<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [1]:
!plaidbench keras mobilenet

Running 1024 examples with mobilenet, batch size 1, on backend plaid
INFO:plaidml:Opening device "opencl_amd_radeon_pro_580_compute_engine.0"
Compiling network... Warming up... Running...
Example finished, elapsed: 0.429s (compile), 9.740s (execution)

-----------------------------------------------------------------------------------------
Network Name         Inference Latency         Time / FPS          
-----------------------------------------------------------------------------------------
mobilenet            9.51 ms                   4.42 ms / 226.43 fps
Correctness: PASS, max_error: 1.675534622336272e-05, max_abs_error: 7.674098014831543e-07, fail_ratio: 0.0


In [2]:
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ["PLAIDML_EXPERIMENTAL"] = "1"
os.environ["PLAIDML_DEVICE_IDS"] = "opencl_amd_radeon_pro_580_compute_engine.0"

Use the following cells to download the data.

In [ ]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
#!unzip concrete_data_week4.zip

In [4]:
!pip install pandas

     |████████████████████████████████| 10.6 MB 2.2 MB/s 
You should consider upgrading via the '/Users/kyle/Documents/github/FineSketch/IBM-Profession-Data-Science/plaidml-venv/bin/python3 -m pip install --upgrade pip' command.


In [6]:
!pip install matplotlib

     |████████████████████████████████| 11.4 MB 1.2 MB/s 
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 60 kB 13.1 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
You should consider upgrading via the '/Users/kyle/Documents/github/FineSketch/IBM-Profession-Data-Science/plaidml-venv/bin/python3 -m pip install --upgrade pip' command.


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, AveragePooling2D, Flatten, Dropout
from keras.models import load_model

#from keras.applications import ResNet50
#from keras.applications.resnet50 import preprocess_input

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator


Using plaidml.keras.backend backend.


In [8]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [9]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [10]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training)

Found 30001 images belonging to 2 classes.


In [11]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation)

Found 9501 images belonging to 2 classes.


In [12]:
model=Sequential()
model.add(VGG16(weights='imagenet',include_top=False,pooling='avg'))
model.layers[0].trainable = False

model.add(Dense(256,activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

INFO:plaidml:Opening device "opencl_amd_radeon_pro_580_compute_engine.0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 14,846,530
Trainable params: 131,842
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [15]:
fit_history = model.fit_generator(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator,
)

model.save('classifier_vgg16_model.h5')

Epoch 1/2
301/301 [==============================] - 4306s 14s/step - loss: nan - acc: 0.5002 - val_loss: nan - val_acc: 0.5001
Epoch 2/2
301/301 [==============================] - 4309s 14s/step - loss: nan - acc: 0.4984 - val_loss: nan - val_acc: 0.5001


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
# 1. Load your saved model that was built using the ResNet50 model.

from keras.models import load_model
model_resnet = load_model('classifier_resnet_model.h5')


In [ ]:
# 2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the shuffle parameter and set it to False.

test_datagen_resnet = ImageDataGenerator()
validation_generator_resnet = test_datagen_resnet.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

In [ ]:
# 3. Use the evaluate_generator method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about evaluate_generator here.

score_resnet = model_resnet.evaluate_generator(
    validation_generator_resnet, 
    workers=0)

In [ ]:
# 4. Print the performance of the classifier using the VGG16 pre-trained model.

model_vgg16 = load_model('classifier_vgg16_model.h5')

test_datagen_vgg16 = ImageDataGenerator()
validation_generator_vgg16 = test_datagen_vgg16.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

score_vgg16 = model_vgg16.evaluate_generator(
    validation_generator_vgg16, 
    workers=0)    

score_vgg16

In [ ]:
print('VGG16 test accuracy:', (score_vgg16[1]*100.0))

In [ ]:
# 5. Print the performance of the classifier using the ResNet pre-trained model.

score_resnet

print('ResNet test accuracy:', (score_resnet[1]*100.0))

In [ ]:
score_resnet

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
predict_vgg16 = model_vgg16.predict_generator(
    validation_generator_vgg16, 
    workers=12)

predict_vgg16     

In [ ]:
correct = 0
for i, n in enumerate(validation_generator_vgg16.filenames):
    print(n)
    print(predict_vgg16[i][0])
    if i == 10:
        break



### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).